In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install arch

import pandas as pd
from arch import arch_model
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score, confusion_matrix
)

import warnings
from arch.univariate.base import ConvergenceWarning, DataScaleWarning

warnings.filterwarnings("ignore", category=DataScaleWarning)
warnings.filterwarnings("ignore", category=ConvergenceWarning)


# Load & compute lagged return
file_path = '/content/drive/MyDrive/MRP/final_dataset.csv'
df = pd.read_csv(file_path, parse_dates=['date'])
df = df.sort_values(['symbol','date']).reset_index(drop=True)

# compute 1-day lagged return and drop the NaNs it creates
df['return_1d_lag1'] = df.groupby('symbol')['return_1d'].shift(1)
df = df.dropna(subset=['return_1d_lag1']).reset_index(drop=True)

# Chronological train/val/test splits
train = df[df['date'] <= '2021-12-31']
val   = df[(df['date'] > '2021-12-31') & (df['date'] <= '2022-12-31')]
test  = df[df['date'] > '2022-12-31']

# Container for per-symbol forecasts
all_preds = []

# Loop over each stock symbol
for sym, grp in df.groupby('symbol'):
    # extract the symbol-specific lagged series from training set
    train_ser = train[train['symbol'] == sym]['return_1d_lag1']
    # identify test indices and true targets
    test_idx  = test[test['symbol'] == sym].index
    true_tgt  = test.loc[test_idx, 'target']

    # skip symbols with too little history or no test points
    if len(train_ser) < 30 or len(test_idx) == 0:
        continue

    try:
        # specify & fit GARCH(1,1) with AR(1) mean on the lagged series
        am  = arch_model(
            train_ser,
            mean='AR', lags=1,
            vol='GARCH', p=1, q=1,
            dist='normal',
            rescale=False
        )
        res = am.fit(disp='off', show_warning=False)

        # multi-step forecast out to the test horizon
        fcast      = res.forecast(horizon=len(test_idx), reindex=False)
        mean_preds = fcast.mean.iloc[0].values  # forecast of return_1d_lag1

        # assemble a per-symbol DataFrame
        preds = pd.DataFrame({
            'symbol'     : sym,
            'date'       : test.loc[test_idx, 'date'],
            'pred_return': mean_preds,
            'true_target': true_tgt
        }, index=test_idx)

        all_preds.append(preds)

    except Exception:
        # skip any symbol where the model fails to converge
        continue

# aggregate & binarize
pred_df = pd.concat(all_preds).sort_index()
pred_df['pred_target'] = (pred_df['pred_return'] > 0).astype(int)

# compute classification metrics on test set
y_true = pred_df['true_target']
y_pred = pred_df['pred_target']

print("GARCH(1,1)+AR(1) on lagged-return series (no leakage):")
print(f"  Accuracy      : {accuracy_score(y_true, y_pred):.4f}")
print(f"  Precision     : {precision_score(y_true, y_pred):.4f}")
print(f"  Recall        : {recall_score(y_true, y_pred):.4f}")
print(f"  F1 Score      : {f1_score(y_true, y_pred):.4f}")
print(f"  ROC AUC       : {roc_auc_score(y_true, pred_df['pred_return']):.4f}")
print("  Confusion Matrix:")
print(confusion_matrix(y_true, y_pred))

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.3/985.3 kB 12.3 MB/s eta 0:00:00


/usr/local/lib/python3.11/dist-packages/arch/univariate/mean.py:1000: RuntimeWarning: overflow encountered in square
  lrf = var_fcasts[:, : (i + 1)].dot(impulse[i::-1] ** 2)


GARCH(1,1)+AR(1) on lagged-return series (no leakage):
  Accuracy      : 0.5106
  Precision     : 0.5110
  Recall        : 0.6649
  F1 Score      : 0.5779
  ROC AUC       : 0.5115
  Confusion Matrix:
[[ 99230 181243]
 [ 95436 189386]]
